## GEO 877 - Spatial Algorithms 
### Group Kirsteina 
#### Tamara, Joya, Andrejs, Djordje

### Data MAKING AN EDIT IN THE .IPYNB FILE :)


#### Description
For our analysis, we will be using the following data:
- Park data - Grünfläschen - Stadt Zürich - **BOUNDARY DATA**
    - 4 data types: Parks, Sports Areas, Cemeteries, Other
    - We will use Parks, Sports Areas, and Cemeteries as designated "parks"
    - We are considering adding Forests to the "parks" set
        - Could someone please download it, I can't: https://www.stadt-zuerich.ch/geodaten/download/111
    - **Can we find a shapefile somewhere that has just green areas as polygons?** we can use it to calculate % areas


- Fountain data - **EVALUATION DATA**
    - Brunnen - fountains for heat relief
    - Stillgewässer - fountains for drinking 
    - **Can we find a shapefile somewhere that has just water areas as polygons?** we can use it to calculate % areas


- ZüriWC data - **EVALUATION DATA**
    - Location of publicly accessible WCs


- Spielpark data - **EVALUATION DATA**
    - Location of kids' playgrounds


- LIDAR data - Canopy Height - **EVALUATION DATA**
    - Download link: https://www.stadt-zuerich.ch/geodaten/download/Baumhoehen_2022__CHM_aus_Lidar_ 


- Socialshilfe data - **EVALUATION DATA**
    - Data on Social Assistance Quotas - need additional clarity whether this is percent of residents that receive Social Assistance or else


#### Data Structure

| PolygonID | Polygon Info | Green Area | Water Area | WC | Fountain | Playground | Socialshilfe | Canopy Height |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| X1 | coordinates | % of park | % of park | yes/no boolean | yes/no boolean | yes/no boolean | quota* | height* |

*needs more thought and discussion


### Analysis Workflow - Preliminary


1. Create park polygons:
    1. Download Wald data
    2. Remove "other" category
    3. Add Wald data
    4. Finalize all polygons - ensure all lines are cohesive
2. LIDAR Data:
    1. Prepare LIDAR data
    2. Extract only relevant data for Parks (Union of Park extent and LIDAR) to remove excess
3. Calculations:
    1. Park area as green space - %
    2. Park area as water space - %
4. Data Merge:
    1. Green area - join data to each park
    2. Water area - join data to each park
    3. Socialshilfe - join data to each park
5. Point in Polygon:
    1. WC - each park receives a numeric value
    2. Fountain - each park receives a numeric value
    3. Playground - each park receives a numeric value
6. Point in Polygon - translated:
    1. For each (WC, Fountain, Playground) assign a yes/no as result of a logical test loop (0 = no, 0> yes)
7. LIDAR Data
    1. **Determine a way to use this data** that is in line with the paper
8. Clustering


### Production

#### 1. Polygon creation (could it be done algorithmically?)

#### 2. LiDAR data preparation

#### 3. % Space Calculations (I think that we can make this an algorithm, too)

#### 4. Data Merging/Joining

#### 5. Point in Polygon (Algorithm)

#### X. Clustering (We could attempt an algorithm)